In [7]:
import numpy as np
import os
import glob
import pandas as pd

def load_text_numpy(path, delimiter, dtype):
    if isinstance(delimiter, (tuple, list)):
        for d in delimiter:
            try:
                ground_truth_rect = np.loadtxt(path, delimiter=d, dtype=dtype)
                return ground_truth_rect
            except:
                pass

        raise Exception('Could not read file {}'.format(path))
    else:
        ground_truth_rect = np.loadtxt(path, delimiter=delimiter, dtype=dtype)
        return ground_truth_rect

def load_text(path, delimiter=' ', dtype=np.float32, backend='numpy'):
    if backend == 'numpy':
        return load_text_numpy(path, delimiter, dtype)


folder = "Dump Annotations Test"
file_dir = os.path.join(os.path.abspath(os.getcwd()), folder, "*.zip")
names = glob.glob(file_dir)
names_anno = [name.split('/')[-1].split('.')[0] for name in names]
names_excel = ['_'.join(name.split('_')[1:-1]) for name in names_anno]
names_excel.sort()
names_pred = names_excel


In [14]:
tracker = 'DIMP50'

In [15]:
from scipy import stats
items = []
x_corrs =[]
y_corrs =[]
for i in range(len(names_pred)):
    item = []
    item.append(names_pred[i])
    anno_name = names_pred[i] + '.csv'
    anno_path = os.path.join(os.path.abspath(os.getcwd()), 'anno_csv', anno_name)
    df_anno = pd.read_csv(anno_path, header = None)
    file_name = names_pred[i] + '.txt'
    results_path = os.path.join(os.path.abspath(os.getcwd()), tracker, file_name) 
    pred_bb = load_text(str(results_path), delimiter=('\t', ','), dtype=np.float64)
    pred_center = pred_bb[:, :2] + 0.5 * (pred_bb[:, 2:] - 1.0)
    df_pred = pd.DataFrame(pred_center)

    x_anno = df_anno[0].to_numpy()
    x_pred = df_pred[0].to_numpy()
    y_anno = df_anno[1].to_numpy()
    y_pred = df_pred[1].to_numpy()
    x_corr, _ = stats.pearsonr(x_anno, x_pred)
    y_corr, _ = stats.pearsonr(y_anno, y_pred)
    item.append(x_corr)
    item.append(y_corr)
    items.append(item)
    x_corrs.append(x_corr)
    y_corrs.append(y_corr)

In [16]:
x_corrs = np.array(x_corrs)
y_corrs = np.array(y_corrs)
x_mean = np.mean(x_corrs)
x_std = np.std(x_corrs)
y_mean = np.mean(y_corrs)
y_std = np.std(y_corrs)
print('mean_x', 'std_x')
print(x_mean, x_std)
print('mean_y', 'std_y')
print(y_mean, y_std)

mean_x std_x
0.9793893278987372 0.015694508920145102
mean_y std_y
0.8898533928321521 0.12257828773777336


In [64]:
df = pd.DataFrame(items, columns =['name', 'x_corr', 'y_corr'])

In [65]:
df.to_excel(tracker + '.xlsx')